In [1]:
import random
import json
import pickle

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

import numpy as np

import speech_recognition as sr
import pyttsx3
import time

In [2]:
lemmatizer = WordNetLemmatizer()
 
queries = json.loads(open('queries.json').read())
 
words = []
classes = []
documents = []
 
ignore_letters = ['?', '!', '.', ',']
 
for i in queries['intents']:
    for j in i['patterns']:
        
        word_list = nltk.word_tokenize(j)
        words.extend(word_list)
        documents.append((word_list, i['tag']))
 
        if i['tag'] not in classes:
            classes.append(i['tag'])
            
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
 
words = sorted(set(words))
classes = sorted(set(classes))

In [3]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [4]:
dataset = []
template = [0]*len(classes)
 
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
 
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
 
    output_row = list(template)
    output_row[classes.index(document[1])] = 1
    dataset.append([bag, output_row])

random.shuffle(dataset)
dataset = np.array(dataset, dtype=object)
 
train_x = list(dataset[:, 0])
train_y = list(dataset[:, 1])

In [5]:
model = Sequential()

model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(train_y[0]), activation='softmax'))
 
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
 
model.save('chatbot_veda.h5', hist)
print('Done!')

C:\Users\rohit\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\rohit\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.3647 - loss: 0.7429
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7659 - loss: 0.6133 
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4384 - loss: 0.6845  
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5820 - loss: 0.6552 
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7522 - loss: 0.5994 
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5557 - loss: 0.6611 
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5988 - loss: 0.6719 
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6357 - loss: 0.6364 
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6722 - loss: 0.5956 
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9263 - loss: 0.5129 
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8094 - loss: 0.5079 
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9031 - lo

Done!


In [6]:
def predict_class(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    res = model.predict(np.array([bag]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return classes[results[0][0]]

In [7]:
def get_response(text, queries):
    for i in queries['intents']:
        for pattern in i['patterns']:
            if pattern in text.lower():
                return i['responses'][pattern]
    return 'sorry, I didn\'t understand.'

In [8]:
def calling_the_bot(text):
    global res
    predict = predict_class(text)
 
    if predict:
        res = get_response(text, queries)
        engine.say('Found it. From our database we found that ' + res)

    engine.runAndWait()
    print('Your Symptom was: ', text)
    print('Result found in our Database: ', res)

In [9]:
if __name__ == '__main__':
    print('Veda the bot, is activated')
 
    recognizer = sr.Recognizer()
    mic = sr.Microphone()
 
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')
 
    engine.setProperty('rate', 175)
 
    volume = engine.getProperty('volume')
    engine.setProperty('volume', 1.0)
 
    voices = engine.getProperty('voices')
 
    engine.say('Hello! I am Veda, your healthcare assistance chatbot.')
    engine.runAndWait()
 
    #engine.setProperty('voice', voices[1].id)
 
    while True or final.lower() == 'yes':
        
        with mic as symptom:
            print('please tell me your symptoms')
            engine.say('I am listening, please tell me your symptoms.')
            engine.runAndWait()
            
            try:
                recognizer.adjust_for_ambient_noise(symptom, duration=0.2)
                symp = recognizer.listen(symptom)
                text = recognizer.recognize_google(symp)
                engine.say('You said {}'.format(text))
                engine.runAndWait()
 
                engine.say('scanning our database for your symptom, please do wait.')
                engine.runAndWait()
 
                time.sleep(1)
 
                calling_the_bot(text)
    
            except sr.UnknownValueError:
                engine.say('sorry, either your symptom is unclear to me \
                    or it is not present in our database, please try again.')
                engine.runAndWait()

            finally:
                engine.say('if you want to continue please say yes otherwise say quit.')
                engine.runAndWait()
 
        with mic as ans:
            try:
                recognizer.adjust_for_ambient_noise(ans, duration=0.2)
                voice = recognizer.listen(ans)
                final = recognizer.recognize_google(voice)
                
            except sr.UnknownValueError:
                engine.say('I could not understand your response, could you please repeat.')
                engine.runAndWait()
                recognizer.adjust_for_ambient_noise(ans, duration=0.2)
                voice = recognizer.listen(ans)
                final = recognizer.recognize_google(voice)
                continue
 
        if final.lower() == 'quit':
            print('Do you want to continue? :',final.lower())
            engine.say('Thank You. At your assistance anytime.')
            engine.runAndWait()
            print('Veda the bot has been deactivated.')
            break

Veda the bot, is activated
please tell me your symptoms
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Your Symptom was:  head pounding and throbbing
Result found in our Database:  migraine
Do you want to continue? : quit
Veda the bot has been deactivated.
